### 🎯 Module Overview
This module covers everything you need to know about parsing and ingesting data for RAG systems, from basic text files to complex PDFs and databases. We'll use LangChain v0.3 and explore each technique with practical examples.

Table of Contents

- Introduction to Data Ingestion
- Text Files (.txt)
- PDF Documents
- Microsoft Word Documents
- CSV and Excel Files
- JSON and Structured Data
- Web Scraping
- Databases (SQL)
- Audio and Video Transcripts
- Advanced Techniques
- Best Practices

In [33]:
import os
from typing import List, Dict, Any
import pandas as pd

In [34]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print("Set up Completed!")


Set up Completed!


## Understanding Document Structure in Langchain

In [35]:
## Create a simple document

doc = Document(
   page_content = "This is the main text content that will be embedded and searched.",
   Meta_data = {
      "Source":"example.txt",
      "Page": 1,
      "Author":"Alka Singh",
      "Date_Created":"2026-02-22",
      "Custom_field":"Any_value"
   }
)
print("Documnet Structure")
print(f"Content:{doc.page_content}")
print(f"Metadata:{doc.metadata}")

## Why Metadata parameters

print("\n Metadata is crucial for: ")
print("- Filtering serach result")
print("- Tracking document sources")
print("- Providing context in responses")
print("- Providing and auditing")

Documnet Structure
Content:This is the main text content that will be embedded and searched.
Metadata:{}

 Metadata is crucial for: 
- Filtering serach result
- Tracking document sources
- Providing context in responses
- Providing and auditing


In [36]:
type(doc)

langchain_core.documents.base.Document

### Text Files(.txt) - The Simplest Case(#2-Case-Files)

In [48]:
import os
os.makedirs("data/text_files", exist_ok = True)

In [49]:
sample_texts = {
   
   "data/text_files/python_intro.txt": """Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """
}


for filepath, content in sample_texts.items():
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created")




✅ Sample text files created


In [50]:
import os
print(os.getcwd())

/Users/alkasingh/Desktop/RAG Project/RAG_Udemy/RAG_Udemy/Data Ingestion and Data Parsing Tech


In [51]:
for filepath, content in sample_texts.items():
   print(filepath)

data/text_files/python_intro.txt
data/text_files/machine_learning.txt


## TextLoader - Read Single File

In [55]:
# from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

## Loading a single text file

loader = TextLoader("data/text_files/python_intro.txt", encoding = "utf-8")

documents = loader.load()

print(f"📄 Loaded {len(documents)} document")
print(f"Content preview: {documents[0].page_content[:100]}...")
print(f"Metadata: {documents[0].metadata}")

📄 Loaded 1 document
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Metadata: {'source': 'data/text_files/python_intro.txt'}


## DirectoryLoader - Multiple Text Files

In [58]:

from langchain_community.document_loaders import DirectoryLoader

## load all the text files from directory

dir_loader = DirectoryLoader(
   "data/text_files",
   glob = "**/*.txt", # patterns to match files
   loader_cls = TextLoader, # Loader class to use
   loader_kwargs = {'encoding':'utf-8'},
   show_progress = True

)

documents = dir_loader.load()

print(f"📄 Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
   print(f"\nDocument {i+1}:")
   print(f" Source: {doc.metadata['source']}")
   print(f" Length: {len(doc.page_content)} characters")


# 📊 Analysis

print("\n📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print("- Load multiple files at once")
print("- Support glob patterns")
print("- Progress tracking")
print("- Recursive directory scanning")

print("\n❌ Disadvantages:")
print("- All files must be of same type")
print("- Limited error handling per file")
print("- Can be memory intensive for large directories")

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 335.13it/s]

📄 Loaded 2 documents

Document 1:
 Source: data/text_files/python_intro.txt
 Length: 489 characters

Document 2:
 Source: data/text_files/machine_learning.txt
 Length: 575 characters

📊 DirectoryLoader Characteristics:
✅ Advantages:
- Load multiple files at once
- Support glob patterns
- Progress tracking
- Recursive directory scanning

❌ Disadvantages:
- All files must be of same type
- Limited error handling per file
- Can be memory intensive for large directories


## Text Splitting Strategies

In [60]:
## Different test splitting strategies
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print(documents)


[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'), Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learn

### Method1: Character Text Splitter

In [65]:
text = documents[1].page_content
text

'Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '

In [63]:
print("1️⃣ CHARACTER TEXT SPLITTER")

char_splitter = CharacterTextSplitter(
   separator = " ",        # Split on newlines
   chunk_size = 200,       # Max chunk size in characters
   chunk_overlap = 20,     # Overlap between chunks
   length_function = len   # How to measure chunk size
)

char_chunks = char_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}...")

1️⃣ CHARACTER TEXT SPLITTER
Created 3 chunks
First chunk: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...


In [64]:
print(char_chunks[0])
print("------------------")
print(char_chunks[1])

Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has
------------------
in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong


In [66]:
print(sample_texts)

{'data/text_files/python_intro.txt': 'Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.', 'data/text_files/machine_learning.txt': 'Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabele

In [67]:
print("1️⃣ CHARACTER TEXT SPLITTER")

char_splitter = CharacterTextSplitter(
   separator = "\n",        # Split on newlines
   chunk_size = 200,       # Max chunk size in characters
   chunk_overlap = 20,     # Overlap between chunks
   length_function = len   # How to measure chunk size
)

char_chunks = char_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}...")

1️⃣ CHARACTER TEXT SPLITTER
Created 4 chunks
First chunk: Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems...


In [68]:
print(char_chunks[0])
print("------------------")
print(char_chunks[1])
print("------------------")
print(char_chunks[2])

Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
------------------
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.
Types of Machine Learning:
------------------
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties


In [76]:
# Method 2: Recursive Charater splitting

print("\n2️⃣Recursive Character Text Splitter")

simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."
splitter = RecursiveCharacterTextSplitter(
   separators = [" "],
   chunk_size = 80,
   chunk_overlap = 20,
   length_function = len
)

chunk = splitter.split_text(simple_text)

print(f"\n Simple text example - {len(chunks)} chunks:\n")

for i in range(len(chunk) - 1):
   print(f"Chunk {i+1}: '{chunk[i]}'")
   print(f"Chunk {i+2}: '{chunk[i+1]}'")

   print()



2️⃣Recursive Character Text Splitter

 Simple text example - 4 chunks:

Chunk 1: 'This is sentence one and it is quite long. This is sentence two and it is also'
Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'

Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'
Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'

Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'
Chunk 4: 'is sentence five. This is sentence six.'



In [80]:
# Method 3: Token Text Splitter

print("\n2️⃣ Token Text Splitter")
print(text)

token_splitter = TokenTextSplitter(
   chunk_size = 50,
   chunk_overlap = 10
)

token_chunk = token_splitter.split_text(text)
print(f"Created: {len(token_chunk)} chunks")
print(f"First_chunk: {char_chunks[0][:100]}...")


2️⃣ Token Text Splitter
Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


    
Created: 3 chunks
First_chunk: Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems...


In [81]:
# 📊 Comparison
print("\n📊 Text Splitting Methods Comparison:")
print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  Use when: Working with token-limited models")


📊 Text Splitting Methods Comparison:

CharacterTextSplitter:
  ✅ Simple and predictable
  ✅ Good for structured text
  ❌ May break mid-sentence
  Use when: Text has clear delimiters

RecursiveCharacterTextSplitter:
  ✅ Respects text structure
  ✅ Tries multiple separators
  ✅ Best general-purpose splitter
  ❌ Slightly more complex
  Use when: Default choice for most texts

TokenTextSplitter:
  ✅ Respects model token limits
  ✅ More accurate for embeddings
  ❌ Slower than character-based
  Use when: Working with token-limited models
